# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [80]:
import requests
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/Philipklein-rodick/.secret/yelp_api.json")

api_key = keys['api_key']

term = 'pizza'
location = 'Buffalo Grove IL'
SEARCH_LIMIT = 50

url = 'https://api.yelp.com/v3/businesses/search'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': 0
            }
response = requests.get(url, headers=headers, params=url_params)

In [67]:
type(response.json()['businesses'][0])

dict

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [83]:
def getall(url, headers, url_params):
    counter = 0
    offset = 50
    url_params['offset'] = 0
    list_of_restaurants = []
    response = requests.get(url, headers = headers, params = url_params)
    list_of_restaurants+=(response.json()["businesses"])
    while (len(response.json()['businesses']) == 50):
        url_params['offset']+=offset
        response = requests.get(url, headers = headers, params = url_params)
        list_of_restaurants+=(response.json()["businesses"])     
        counter+=1
        if counter>100:
            break
    return list_of_restaurants
restaurants = getall(url, headers, url_params)
len(restaurants)

176

In [86]:
import pandas as pd
df = pd.DataFrame(restaurants)
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,lou-malnatis-pizzeria-buffalo-grove,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 42.1528720667073, 'longitude': -8...",(847) 215-7100,1120.543521,N6YgYTC59AxnYHMoQWRRyA,https://s3-media4.fl.yelpcdn.com/bphoto/vBQQHr...,False,"{'address1': '85 S Buffalo Grove Rd', 'address...",Lou Malnati's Pizzeria,+18472157100,$$,4.0,444,"[delivery, pickup]",https://www.yelp.com/biz/lou-malnatis-pizzeria...
1,ninos-pizzeria-and-catering-buffalo-grove,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 42.137794, 'longitude': -87.959296}",(847) 947-4111,2789.909018,p7yUtI6EKuhzjps00mITHg,https://s3-media4.fl.yelpcdn.com/bphoto/Y0Czbs...,False,"{'address1': '960 S Buffalo Grove Rd', 'addres...",Nino's Pizzeria & Catering,+18479474111,$$,4.0,217,[],https://www.yelp.com/biz/ninos-pizzeria-and-ca...
2,giordanos-buffalo-grove,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 42.157985687121, 'longitude': -87...",(847) 520-1600,593.933066,avKXTmRWhnttRApH6pCKDw,https://s3-media4.fl.yelpcdn.com/bphoto/rFw80P...,False,"{'address1': '270 N McHenry Rd', 'address2': '...",Giordano's,+18475201600,$$,3.5,106,"[delivery, pickup]",https://www.yelp.com/biz/giordanos-buffalo-gro...
3,pizanoz-pizza-buffalo-grove,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 42.1720862, 'longitude': -87.9493...",(847) 634-3380,1324.693185,tIhPS-rsWn5EnpjOW3VRGg,https://s3-media4.fl.yelpcdn.com/bphoto/NoCT2R...,False,"{'address1': '1137 Weiland Rd', 'address2': ''...",Pizanoz Pizza,+18476343380,$,3.5,52,"[delivery, pickup]",https://www.yelp.com/biz/pizanoz-pizza-buffalo...
4,bonta-lincolnshire-4,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","{'latitude': 42.1991299, 'longitude': -87.93302}",(847) 383-6233,4554.482478,pDr9PC7jD7LBnLgJxD_4wQ,https://s3-media3.fl.yelpcdn.com/bphoto/Wi_eTw...,False,"{'address1': '430 Milwaukee Ave', 'address2': ...",Bonta,+18473836233,$$,4.5,399,"[delivery, pickup]",https://www.yelp.com/biz/bonta-lincolnshire-4?...


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [99]:
df[['price', 'rating', 'review_count']].head()

,price,rating,review_count
0,$$,4.0,444
1,$$,4.0,217
2,$$,3.5,106
3,$,3.5,52
4,$$,4.5,399


In [97]:
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,lou-malnatis-pizzeria-buffalo-grove,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 42.1528720667073, 'longitude': -8...",(847) 215-7100,1120.543521,N6YgYTC59AxnYHMoQWRRyA,https://s3-media4.fl.yelpcdn.com/bphoto/vBQQHr...,False,"{'address1': '85 S Buffalo Grove Rd', 'address...",Lou Malnati's Pizzeria,+18472157100,$$,4.0,444,"[delivery, pickup]",https://www.yelp.com/biz/lou-malnatis-pizzeria...
1,ninos-pizzeria-and-catering-buffalo-grove,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 42.137794, 'longitude': -87.959296}",(847) 947-4111,2789.909018,p7yUtI6EKuhzjps00mITHg,https://s3-media4.fl.yelpcdn.com/bphoto/Y0Czbs...,False,"{'address1': '960 S Buffalo Grove Rd', 'addres...",Nino's Pizzeria & Catering,+18479474111,$$,4.0,217,[],https://www.yelp.com/biz/ninos-pizzeria-and-ca...
2,giordanos-buffalo-grove,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 42.157985687121, 'longitude': -87...",(847) 520-1600,593.933066,avKXTmRWhnttRApH6pCKDw,https://s3-media4.fl.yelpcdn.com/bphoto/rFw80P...,False,"{'address1': '270 N McHenry Rd', 'address2': '...",Giordano's,+18475201600,$$,3.5,106,"[delivery, pickup]",https://www.yelp.com/biz/giordanos-buffalo-gro...
3,pizanoz-pizza-buffalo-grove,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 42.1720862, 'longitude': -87.9493...",(847) 634-3380,1324.693185,tIhPS-rsWn5EnpjOW3VRGg,https://s3-media4.fl.yelpcdn.com/bphoto/NoCT2R...,False,"{'address1': '1137 Weiland Rd', 'address2': ''...",Pizanoz Pizza,+18476343380,$,3.5,52,"[delivery, pickup]",https://www.yelp.com/biz/pizanoz-pizza-buffalo...
4,bonta-lincolnshire-4,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","{'latitude': 42.1991299, 'longitude': -87.93302}",(847) 383-6233,4554.482478,pDr9PC7jD7LBnLgJxD_4wQ,https://s3-media3.fl.yelpcdn.com/bphoto/Wi_eTw...,False,"{'address1': '430 Milwaukee Ave', 'address2': ...",Bonta,+18473836233,$$,4.5,399,"[delivery, pickup]",https://www.yelp.com/biz/bonta-lincolnshire-4?...


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [103]:
import folium
lat = 42.13
long = -87.9592
base_map = folium.Map([lat, long], zoom_start=13)

for row in df.index:
    lat_long = df['coordinates'][row]
    lat = lat_long['latitude']
    long = lat_long['longitude']
    name = df['name'][row]
    rating = df['rating'][row]
    price = df['price'][row]
    details = "{}\nPrice: {} Rating:{}".format(name,str(price),str(rating))
    popup = folium.Popup(details, parse_html=True)
    marker = folium.Marker([lat, long], popup=popup)
    marker.add_to(base_map)
base_map


## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!